In [36]:
# Importing useful packages
import numpy as np
from scipy import stats
import pandas as pd
import sklearn as sk
import seaborn as sb
import datetime as dt
import pylab 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from new_var import calc,y1function,y2function,C3function,C6function
from outliers import outlier
from Standardising import standard
from Recoding_SIC_Codes import Industry_Division 
%matplotlib inline
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support


# Read in Data file and define NaN values
ipo_data = pd.read_excel("Competition #1 Raw Data_UPDATED I3.xlsx",header=0,na_values='-' )
ipo_data.I3 = ipo_data.I3.astype(object) # Converting to object for the moment to tidy up summary statistics
# Run Descriptive Statistics on Data File
#print(ipo_data.dtypes)
# Replacing NaN in C2 with 1 
ipo_data['C2']=ipo_data['C2'].fillna(1)


# Removing negative and 0 values for the word/sentence counts as it is beleived that these are errors
# Removing 0 and negatives for T' calculations        
ipo_data['T2']=ipo_data['T2'].mask(ipo_data['T2'] <= 0,ipo_data['T2'].mean())
ipo_data['T1']=ipo_data['T1'].mask(ipo_data['T1'] <= 0,ipo_data['T1'].mean())
ipo_data['T3']=ipo_data['T3'].mask(ipo_data['T3'] <= 0,ipo_data['T3'].mean())
ipo_data['T4']=ipo_data['T4'].mask(ipo_data['T4'] <= 0,ipo_data['T4'].mean())
ipo_data['T5']=ipo_data['T5'].mask(ipo_data['T5'] <= 0,ipo_data['T5'].mean())
ipo_data['S1']=ipo_data['S1'].mask(ipo_data['S1'] <= 0,ipo_data['S1'].mean())

# Replace NaN values with mean
ipo_data=ipo_data.fillna(ipo_data.mean())


In [37]:

# Creating our target and control variables
ipo_data['Y1'] = ipo_data.apply(y1function, axis=1)
ipo_data['Y2'] = ipo_data.apply(y2function, axis=1)
ipo_data['C3x'] = ipo_data.apply(C3function, axis=1)
ipo_data['C6x'] = ipo_data.apply(C6function, axis=1)

#Creating ratios for the words, sentences and postive/negative counts. Ratios provide insight
calc(ipo_data)

#Removing redundent data
del ipo_data['C3']
del ipo_data['C5']
del ipo_data['C6']
# del ipo_data['T1']
# del ipo_data['T2']
del ipo_data['T3']
del ipo_data['T4']
del ipo_data['T5']
del ipo_data['S1']
del ipo_data['S2']
del ipo_data['S3']
del ipo_data['P(1Day)']
del ipo_data['P(L)']
del ipo_data['P(H)']
del ipo_data['P(IPO)']
#Looking at the new description of the data
ipo_data.describe()

,C1,C2,C4,C7,T1,T2,Y1,Y2,C3x,C6x,C5x,T3x,T4x,T5x,S1x,S2x,S3x
count,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000,682.000000
mean,149.728788,0.863636,0.007282,500.459962,465.634361,12796.076671,0.513196,0.708211,0.536657,0.052708,4.627711,0.918942,0.629876,0.080692,0.005504,0.009179,0.011854
std,150.328785,0.343426,0.032775,1558.767222,175.612567,5401.439089,0.500193,0.454919,0.499020,0.107639,6.245584,0.624282,0.076265,0.744766,0.004784,0.004324,0.008970
min,10.000000,0.000000,-0.162352,0.074000,132.000000,527.000000,0.000000,0.000000,0.000000,0.000000,0.283223,0.703925,0.395480,0.031595,0.001389,0.002152,0.005125
25%,85.000000,1.000000,-0.013352,41.511500,351.250000,9213.250000,0.000000,0.000000,0.000000,0.000000,2.853439,0.883578,0.580815,0.047413,0.004154,0.007031,0.009570
50%,110.000000,1.000000,0.007282,134.738000,444.500000,12064.000000,1.000000,1.000000,1.000000,0.000000,3.770051,0.897133,0.629077,0.051901,0.005069,0.008600,0.011305
75%,152.750000,1.000000,0.030941,500.459962,550.750000,15240.250000,1.000000,1.000000,1.000000,0.071429,4.865080,0.909732,0.674852,0.056471,0.006258,0.010535,0.012883
max,2087.000000,1.000000,0.092896,30683.000000,1750.000000,49056.000000,1.000000,1.000000,1.000000,1.000000,99.787255,17.165085,1.443089,19.500949,0.121442,0.091082,0.233397


In [38]:
# Converting C6x and C2 to correct type
ipo_data.C6x = ipo_data.C6x.astype(float)

ipo_data.C2 = ipo_data.C2.astype(int)
#Filing days was broken into three seperate groups. Under six month, 6-12 months, and over 12 months. 
ipo_data['Days'] = 0
ipo_data.loc[(ipo_data['C1'] >= 0) & (ipo_data['C1'] <= 180),'Days'] = 0
ipo_data.loc[(ipo_data['C1'] > 180), 'Days'] = 1



sqrt_transform=['C6x','S1x','S2x','S3x'] #Variables that need to be square rooted
ipo_data[sqrt_transform]=ipo_data[sqrt_transform]**0.5 #square rooting variable

log_transform=['C1','C7'] #Variables to be log transformed
ipo_data[log_transform]=np.log(ipo_data[log_transform])#log transformation

ipo_data['T5xbin'] = 0
# ipo_data.loc[(ipo_data['T5x'] >= 0.080692),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= 0) & (ipo_data['T5x'] < .0474),'T5xbin'] = 1
ipo_data.loc[(ipo_data['T5x'] >= .047401) & (ipo_data['T5x'] < .0519), 'T5xbin'] = 2
ipo_data.loc[(ipo_data['T5x'] >= .051901) & (ipo_data['T5x'] < .05647), 'T5xbin'] = 3
ipo_data.loc[ipo_data[ 'T5x'] >= .0564701,'T5xbin'] = 4

ipo_data['T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] < 0.897133),'T3xbin'] = 0
# ipo_data.loc[(ipo_data['T3x'] >= 0.897133),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= 0.703925) & (ipo_data['T3x'] < 0.883578),'T3xbin'] = 1
ipo_data.loc[(ipo_data['T3x'] >= .883578) & (ipo_data['T3x'] < 0.897133), 'T3xbin'] = 2
ipo_data.loc[(ipo_data['T3x'] >= .897133) & (ipo_data['T3x'] < 0.909732), 'T3xbin'] = 3
ipo_data.loc[ipo_data[ 'T3x'] >= 0.909732,'T3xbin'] = 4

print(ipo_data.dtypes) # checking the types
print(ipo_data.T5xbin.value_counts())

I1         object
I2         object
I3         object
C1        float64
C2          int32
C4        float64
C7        float64
T1        float64
T2        float64
Y1          int64
Y2          int64
C3x         int64
C6x       float64
C5x       float64
T3x       float64
T4x       float64
T5x       float64
S1x       float64
S2x       float64
S3x       float64
Days        int64
T5xbin      int64
T3xbin      int64
dtype: object
4    171
2    171
3    170
1    170
Name: T5xbin, dtype: int64


In [39]:
"""Code to recode the column I3 into industry level.

    Version Control:
    Initial coding
    ------------------------------------------
    Date 8-Feb-18, Author: Danielle Ezzo, Desc: Initial Coding
    """

def Industry_Division1(row,method):
    # Recoding based on table in milestone report
    if method==1:
        if (row['I3']> 99 and row['I3']< 1000):
            return 'Agriculture, Forestry and Fishing'
        elif (row['I3']> 999 and row['I3']< 1500):
            return  'Mining'
        elif (row['I3']> 1499 and row['I3']< 1800):
            return 'Construction'
        elif (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Transportation, Communications, Electric, Gas and Sanitary service'
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Wholesale Trade'
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Retail Trade'
        elif (row['I3']> 5999 and row['I3']< 6800):
            return 'Finance, Insurance and Real Estate'
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        elif (row['I3']> 9099 and row['I3']< 9730):
            return 'Public Administration'
        elif (row['I3']> 9899 and row['I3']< 10000):
            return 'Nonclassifiable'
        else:
            return 'Missing'
    elif method==2:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        else:
            return 'Other'
    elif method==3:
        if (row['I3']> 7369 and row['I3']< 7375) :
            return 'Tech'
        else:
            return 'NonTech'
    elif method==4:
        if (row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing' # Manufacturing
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Services' # Tech Comms etc Services
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services' #Services (combining with above)
        else:
            return 'Other' # All Other
    elif method==5:
        if(row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'Services' #TCEGS
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Trade' #wholesale
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Trade' #retail
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        else:
            return 'Other'
    elif method==6:
        if(row['I3']> 1999 and row['I3']< 4000):
            return 'Manufacturing'
        elif (row['I3']> 3999 and row['I3']< 5000):
            return 'TCEGS' 
        elif (row['I3']> 4999 and row['I3']< 5200):
            return 'Trade' #wholesale
        elif (row['I3']> 5199 and row['I3']< 6000):
            return 'Trade' #retail
        elif (row['I3']> 6999 and row['I3']< 9000):
            return 'Services'
        else:
            return 'Other'
    elif method==7:
        if (row['I3']> 99 and row['I3']< 5000):
            return 'Manual Labor'
        else: 
            return 'NonManual Labor'
        

In [40]:
# Add the new data to the end of the table
ipo_data.I3 = ipo_data.I3.astype(int) # recoding to correct type

# import functools
# import operator


# applying our function to recode our industry codes to sector level
ipo_data['IndDivision'] = ipo_data.apply(Industry_Division1,method=4, axis=1) 
print(ipo_data.head(5))

     I1                           I2    I3        C1  C2        C4        C7  \
0    RA         RAILAMERICA INC /DE   4011  4.343805   1  0.005264  6.231398   
1     G                 Genpact LTD   8742  4.418841   1 -0.053282  6.426451   
2  HURN  Huron Consulting Group Inc.  8742  5.030438   1  0.024642  4.703150   
3  ICFI     ICF International, Inc.   8742  4.941642   1  0.051895  5.177381   
4  SRVY       GREENFIELD ONLINE INC   8742  4.553877   1 -0.050492  3.253007   

      T1       T2  Y1     ...            T3x       T4x       T5x       S1x  \
0  463.0  13494.0   1     ...       0.865866  0.710583  0.043871  0.082570   
1  641.0  15238.0   1     ...       0.905303  0.561622  0.047710  0.077702   
2  314.0   9689.0   0     ...       0.874290  0.719745  0.052843  0.055644   
3  453.0  11082.0   1     ...       0.909673  0.576159  0.051886  0.075398   
4  409.0  10417.0   1     ...       0.899587  0.650367  0.048094  0.081974   

        S2x       S3x  Days  T5xbin  T3xbin  IndDi

In [41]:
dummies = pd.get_dummies(ipo_data['IndDivision'])

ipo_data = ipo_data.join(dummies)
ipo_data.head(5)

,I1,I2,I3,C1,C2,C4,C7,T1,T2,Y1,...,S1x,S2x,S3x,Days,T5xbin,T3xbin,IndDivision,Manufacturing,Other,Services
0,RA,RAILAMERICA INC /DE,4011,4.343805,1,0.005264,6.231398,463.0,13494.0,1,...,0.082570,0.122955,0.081668,0,1,1,Services,0,0,1
1,G,Genpact LTD,8742,4.418841,1,-0.053282,6.426451,641.0,15238.0,1,...,0.077702,0.086114,0.109887,0,2,3,Services,0,0,1
2,HURN,Huron Consulting Group Inc.,8742,5.030438,1,0.024642,4.703150,314.0,9689.0,0,...,0.055644,0.070385,0.090297,0,3,1,Services,0,0,1
3,ICFI,"ICF International, Inc.",8742,4.941642,1,0.051895,5.177381,453.0,11082.0,1,...,0.075398,0.081162,0.126022,0,2,3,Services,0,0,1
4,SRVY,GREENFIELD ONLINE INC,8742,4.553877,1,-0.050492,3.253007,409.0,10417.0,1,...,0.081974,0.095998,0.109543,0,2,3,Services,0,0,1


In [43]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,6):
    for m in range(1,5):
        x=pd.DataFrame.copy(ipo_data)
        res_hold_f1=[]
        res_hold_auc=[]
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                outlier(x.iloc[:,i],j)             
                if x.iloc[:,i].dtype == float:   
                    x.iloc[:,i]=standard(x.iloc[:,i],m)
        for k in range(2,6):
            for count in range(1,50):


                # Create K-means model and determine euclidian distances for each data point
                train=['C3x','Days','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','C6x','Other','Manufacturing','Services']
                X=x[train]
                kmeans_model = KMeans(n_clusters=k)
                distances = kmeans_model.fit_transform(X)

                # Create scatter plot using labels from K-means model as color
                labels = kmeans_model.labels_


                x['labels'] = labels
                train.append('labels')


                logreg = LogisticRegression()
                #train=['C4','C6x','T4x','S2x']
                #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

                #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
                #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

                X=x[train]

                y=x['Y2']
                rfe = RFE(logreg, 18)
                rfe = rfe.fit(X,y)

                logit_model=sm.Logit(x['Y2'],x[train])
                

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
                oversample = pd.concat([X_train,y_train],axis=1)
                max_size = oversample['Y2'].value_counts().max()

                lst = [oversample]
                for class_index, group in oversample.groupby('Y2'):
                    lst.append(group.sample(max_size-len(group), replace=True))
                X_train = pd.concat(lst)
                y_train=pd.DataFrame.copy(X_train['Y2'])
                del X_train['Y2']
                logreg = LogisticRegression()
                logreg.fit(X_train, y_train)
                y_pred = logreg.predict(X_test)

                kfold = model_selection.KFold(n_splits=5, random_state=7)
                modelCV = LogisticRegression()
                scoring = 'accuracy'
                results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
                logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
                #confusion_matrix = confusion_matrix(y_test, y_pred)
                res_hold_f1.append(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2])
                res_hold_auc.append(logit_roc_auc)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(np.mean(res_hold_f1),np.mean(res_hold_auc)))
       #     if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
               #   print('True')
                #result=logit_model.fit()
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))


#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')


#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.59; AUC 0.59 
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.59; AUC 0.59 
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.59; AUC 0.59 
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.59; AUC 0.59 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Outlier Method 1; Standardise Method 2 Kmeans 2
F1 0.58; AUC 0.53 
Outlier Method 1; Standardise Method 2 Kmeans 3
F1 0.58; AUC 0.53 
Outlier Method 1; Standardise Method 2 Kmeans 4
F1 0.57; AUC 0.53 
Outlier Method 1; Standardise Method 2 Kmeans 5
F1 0.57; AUC 0.53 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


...

Outlier Method 4; Standardise Method 2 Kmeans 2
F1 0.57; AUC 0.52 
Outlier Method 4; Standardise Method 2 Kmeans 3
F1 0.57; AUC 0.52 
Outlier Method 4; Standardise Method 2 Kmeans 4
F1 0.57; AUC 0.52 
Outlier Method 4; Standardise Method 2 Kmeans 5
F1 0.57; AUC 0.52 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.60; AUC 0.59 
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.60; AUC 0.60 
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.60; AUC 0.59 
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.60; AUC 0.59 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Outlier Method 4; Standardise Method 4 Kmeans 2
F1 0.60; AUC 0.59 
Outlier Method 4; Standardise Method 4 Kmeans 3
F1 0.60; AUC 0.60 
Outlier Method 4; Standardise Method 4 Kmeans 4
F1 0.60; AUC 0.60 
Outlier Method 4; Standardise Method 4 Kmeans 5
F1 0.60; AUC 0.59 
Outlier Method 5; Standardise Method 1 Kmeans 2
F1 0.60; AUC 0.60 
Outlier Method 5; Standardise Method 1 Kmeans 3
F1 0.60; AUC 0.59 
Outlier Method 5; Standardise Method 1 Kmeans 4
F1 0.60; AUC 0.59 
Outlier Method 5; Standardise Method 1 Kmeans 5
F1 0.60; AUC 0.59 
Outlier Method 5; Standardise Method 2 Kmeans 2
F1 0.56; AUC 0.51 
Outlier Method 5; Standardise Method 2 Kmeans 3
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 2 Kmeans 4
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 2 Kmeans 5
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 3 Kmeans 2
F1 0.60; AUC 0.59 
Outlier Method 5; Standardise Method 3 Kmeans 3
F1 0.60; AUC 0.59 
Outlier Method 5; Standardise Method 3 Kmeans 4
F1 0.59; AUC 0

In [44]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,6):
    for m in range(1,5):
        x=pd.DataFrame.copy(ipo_data)
        res_hold_f1=[]
        res_hold_auc=[]
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                outlier(x.iloc[:,i],j)             
                if x.iloc[:,i].dtype == float:   
                    x.iloc[:,i]=standard(x.iloc[:,i],m)
        for k in range(2,6):
            for count in range(1,50):


                # Create K-means model and determine euclidian distances for each data point
                train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','C6x','Other','Manufacturing','Services']
                X=x[train]
                kmeans_model = KMeans(n_clusters=k)
                distances = kmeans_model.fit_transform(X)

                # Create scatter plot using labels from K-means model as color
                labels = kmeans_model.labels_


                x['labels'] = labels
                train.append('labels')


                logreg = LogisticRegression()
                #train=['C4','C6x','T4x','S2x']
                #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

                #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
                #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

                X=x[train]

                y=x['Y2']
                rfe = RFE(logreg, 18)
                rfe = rfe.fit(X,y)

                logit_model=sm.Logit(x['Y2'],x[train])
                

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
                oversample = pd.concat([X_train,y_train],axis=1)
                max_size = oversample['Y2'].value_counts().max()

                lst = [oversample]
                for class_index, group in oversample.groupby('Y2'):
                    lst.append(group.sample(max_size-len(group), replace=True))
                X_train = pd.concat(lst)
                y_train=pd.DataFrame.copy(X_train['Y2'])
                del X_train['Y2']
                logreg = LogisticRegression()
                logreg.fit(X_train, y_train)
                y_pred = logreg.predict(X_test)

                kfold = model_selection.KFold(n_splits=5, random_state=7)
                modelCV = LogisticRegression()
                scoring = 'accuracy'
                results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
                logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
                #confusion_matrix = confusion_matrix(y_test, y_pred)
                res_hold_f1.append(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2])
                res_hold_auc.append(logit_roc_auc)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(np.mean(res_hold_f1),np.mean(res_hold_auc)))
       #     if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
               #   print('True')
                #result=logit_model.fit()
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))


#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')


#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.59; AUC 0.59 
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.59; AUC 0.59 
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.59; AUC 0.59 
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.59; AUC 0.59 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Outlier Method 1; Standardise Method 2 Kmeans 2
F1 0.58; AUC 0.53 
Outlier Method 1; Standardise Method 2 Kmeans 3
F1 0.58; AUC 0.53 
Outlier Method 1; Standardise Method 2 Kmeans 4
F1 0.58; AUC 0.53 
Outlier Method 1; Standardise Method 2 Kmeans 5
F1 0.58; AUC 0.54 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


...

Outlier Method 4; Standardise Method 2 Kmeans 2
F1 0.58; AUC 0.54 
Outlier Method 4; Standardise Method 2 Kmeans 3
F1 0.57; AUC 0.53 
Outlier Method 4; Standardise Method 2 Kmeans 4
F1 0.57; AUC 0.53 
Outlier Method 4; Standardise Method 2 Kmeans 5
F1 0.57; AUC 0.53 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.59; AUC 0.59 
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.59; AUC 0.59 
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.59; AUC 0.59 
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.60; AUC 0.59 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Outlier Method 4; Standardise Method 4 Kmeans 2
F1 0.59; AUC 0.59 
Outlier Method 4; Standardise Method 4 Kmeans 3
F1 0.59; AUC 0.59 
Outlier Method 4; Standardise Method 4 Kmeans 4
F1 0.59; AUC 0.59 
Outlier Method 4; Standardise Method 4 Kmeans 5
F1 0.59; AUC 0.59 
Outlier Method 5; Standardise Method 1 Kmeans 2
F1 0.59; AUC 0.59 
Outlier Method 5; Standardise Method 1 Kmeans 3
F1 0.60; AUC 0.60 
Outlier Method 5; Standardise Method 1 Kmeans 4
F1 0.60; AUC 0.59 
Outlier Method 5; Standardise Method 1 Kmeans 5
F1 0.60; AUC 0.59 
Outlier Method 5; Standardise Method 2 Kmeans 2
F1 0.56; AUC 0.52 
Outlier Method 5; Standardise Method 2 Kmeans 3
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 2 Kmeans 4
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 2 Kmeans 5
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 3 Kmeans 2
F1 0.60; AUC 0.60 
Outlier Method 5; Standardise Method 3 Kmeans 3
F1 0.60; AUC 0.60 
Outlier Method 5; Standardise Method 3 Kmeans 4
F1 0.60; AUC 0

In [45]:
rfe = RFE(logreg, 18)
rfe = rfe.fit(X,y)
logit_model=sm.Logit(x['Y2'],x[train])
result=logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.561555
         Iterations 6


In [46]:
# reducing our sample due to in balance between y2=1 and y2=0
for j in range(1,6):
    for m in range(1,5):
        x=pd.DataFrame.copy(ipo_data)
        res_hold_f1=[]
        res_hold_auc=[]
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                outlier(x.iloc[:,i],j)             
                if x.iloc[:,i].dtype == float:   
                    x.iloc[:,i]=standard(x.iloc[:,i],m)
        for k in range(2,6):
            for count in range(1,50):


                # Create K-means model and determine euclidian distances for each data point
                train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','Other','Manufacturing','Services']
                X=x[train]
                kmeans_model = KMeans(n_clusters=k)
                distances = kmeans_model.fit_transform(X)

                # Create scatter plot using labels from K-means model as color
                labels = kmeans_model.labels_


                x['labels'] = labels
                train.append('labels')


                logreg = LogisticRegression()
                #train=['C4','C6x','T4x','S2x']
                #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

                #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
                #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

                X=x[train]

                y=x['Y1']
                rfe = RFE(logreg, 18)
                rfe = rfe.fit(X,y)

                logit_model=sm.Logit(x['Y1'],x[train])
                

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#                 oversample = pd.concat([X_train,y_train],axis=1)
#                 max_size = oversample['Y2'].value_counts().max()

#                 lst = [oversample]
#                 for class_index, group in oversample.groupby('Y2'):
#                     lst.append(group.sample(max_size-len(group), replace=True))
#                 X_train = pd.concat(lst)
#                 y_train=pd.DataFrame.copy(X_train['Y2'])
#                 del X_train['Y2']
                logreg = LogisticRegression()
                logreg.fit(X_train, y_train)
                y_pred = logreg.predict(X_test)

                kfold = model_selection.KFold(n_splits=5, random_state=7)
                modelCV = LogisticRegression()
                scoring = 'accuracy'
                results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
                logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
                #confusion_matrix = confusion_matrix(y_test, y_pred)
                res_hold_f1.append(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2])
                res_hold_auc.append(logit_roc_auc)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(np.mean(res_hold_f1),np.mean(res_hold_auc)))
       #     if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
               #   print('True')
                #result=logit_model.fit()
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))


#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')


#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()

                

C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.57; AUC 0.57 
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.57; AUC 0.57 
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.57; AUC 0.57 
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.57; AUC 0.57 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Outlier Method 1; Standardise Method 2 Kmeans 2
F1 0.53; AUC 0.53 
Outlier Method 1; Standardise Method 2 Kmeans 3
F1 0.52; AUC 0.52 
Outlier Method 1; Standardise Method 2 Kmeans 4
F1 0.52; AUC 0.53 
Outlier Method 1; Standardise Method 2 Kmeans 5
F1 0.52; AUC 0.53 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


...

Outlier Method 4; Standardise Method 2 Kmeans 2
F1 0.53; AUC 0.53 
Outlier Method 4; Standardise Method 2 Kmeans 3
F1 0.53; AUC 0.53 
Outlier Method 4; Standardise Method 2 Kmeans 4
F1 0.53; AUC 0.53 
Outlier Method 4; Standardise Method 2 Kmeans 5
F1 0.53; AUC 0.53 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Outlier Method 4; Standardise Method 3 Kmeans 2
F1 0.58; AUC 0.58 
Outlier Method 4; Standardise Method 3 Kmeans 3
F1 0.58; AUC 0.59 
Outlier Method 4; Standardise Method 3 Kmeans 4
F1 0.58; AUC 0.58 
Outlier Method 4; Standardise Method 3 Kmeans 5
F1 0.58; AUC 0.58 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Outlier Method 4; Standardise Method 4 Kmeans 2
F1 0.53; AUC 0.53 
Outlier Method 4; Standardise Method 4 Kmeans 3
F1 0.53; AUC 0.54 
Outlier Method 4; Standardise Method 4 Kmeans 4
F1 0.53; AUC 0.54 
Outlier Method 4; Standardise Method 4 Kmeans 5
F1 0.53; AUC 0.54 
Outlier Method 5; Standardise Method 1 Kmeans 2
F1 0.54; AUC 0.55 
Outlier Method 5; Standardise Method 1 Kmeans 3
F1 0.54; AUC 0.54 
Outlier Method 5; Standardise Method 1 Kmeans 4
F1 0.54; AUC 0.54 
Outlier Method 5; Standardise Method 1 Kmeans 5
F1 0.54; AUC 0.54 
Outlier Method 5; Standardise Method 2 Kmeans 2
F1 0.53; AUC 0.54 
Outlier Method 5; Standardise Method 2 Kmeans 3
F1 0.53; AUC 0.54 
Outlier Method 5; Standardise Method 2 Kmeans 4
F1 0.53; AUC 0.53 
Outlier Method 5; Standardise Method 2 Kmeans 5
F1 0.53; AUC 0.53 
Outlier Method 5; Standardise Method 3 Kmeans 2
F1 0.59; AUC 0.59 
Outlier Method 5; Standardise Method 3 Kmeans 3
F1 0.58; AUC 0.59 
Outlier Method 5; Standardise Method 3 Kmeans 4
F1 0.58; AUC 0

In [47]:
# reducing our sample due to in balance between y2=1 and y2=0
for m in range(1,5):
    for j in range(1,6):
        x=pd.DataFrame.copy(ipo_data)
        res_hold_f1=[]
        res_hold_auc=[]
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                x.iloc[:,i]=standard(x.iloc[:,i],m)            
                if x.iloc[:,i].dtype == float:
                    outlier(x.iloc[:,i],j) 
        for k in range(2,6):
            for count in range(1,50):


                # Create K-means model and determine euclidian distances for each data point
                train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','Other','Manufacturing','Services']
                X=x[train]
                kmeans_model = KMeans(n_clusters=k)
                distances = kmeans_model.fit_transform(X)

                # Create scatter plot using labels from K-means model as color
                labels = kmeans_model.labels_


                x['labels'] = labels
                train.append('labels')


                logreg = LogisticRegression()
                #train=['C4','C6x','T4x','S2x']
                #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

                #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
                #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

                X=x[train]

                y=x['Y2']
                rfe = RFE(logreg, 18)
                rfe = rfe.fit(X,y)

                logit_model=sm.Logit(x['Y2'],x[train])
                

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
                oversample = pd.concat([X_train,y_train],axis=1)
                max_size = oversample['Y2'].value_counts().max()

                lst = [oversample]
                for class_index, group in oversample.groupby('Y2'):
                    lst.append(group.sample(max_size-len(group), replace=True))
                X_train = pd.concat(lst)
                y_train=pd.DataFrame.copy(X_train['Y2'])
                del X_train['Y2']
                logreg = LogisticRegression()
                logreg.fit(X_train, y_train)
                y_pred = logreg.predict(X_test)

                kfold = model_selection.KFold(n_splits=5, random_state=7)
                modelCV = LogisticRegression()
                scoring = 'accuracy'
                results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
                logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
                #confusion_matrix = confusion_matrix(y_test, y_pred)
                res_hold_f1.append(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2])
                res_hold_auc.append(logit_roc_auc)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(np.mean(res_hold_f1),np.mean(res_hold_auc)))
       #     if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
               #   print('True')
                #result=logit_model.fit()
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))


#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')


#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()

                                       

C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.58; AUC 0.53 
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.58; AUC 0.53 
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.57; AUC 0.53 
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.57; AUC 0.53 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Outlier Method 2; Standardise Method 1 Kmeans 2
F1 0.57; AUC 0.52 
Outlier Method 2; Standardise Method 1 Kmeans 3
F1 0.57; AUC 0.52 
Outlier Method 2; Standardise Method 1 Kmeans 4
F1 0.56; AUC 0.52 
Outlier Method 2; Standardise Method 1 Kmeans 5
F1 0.57; AUC 0.52 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


...

Outlier Method 2; Standardise Method 4 Kmeans 2
F1 0.56; AUC 0.52 
Outlier Method 2; Standardise Method 4 Kmeans 3
F1 0.56; AUC 0.52 
Outlier Method 2; Standardise Method 4 Kmeans 4
F1 0.56; AUC 0.52 
Outlier Method 2; Standardise Method 4 Kmeans 5
F1 0.56; AUC 0.52 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Outlier Method 3; Standardise Method 4 Kmeans 2
F1 0.55; AUC 0.51 
Outlier Method 3; Standardise Method 4 Kmeans 3
F1 0.56; AUC 0.51 
Outlier Method 3; Standardise Method 4 Kmeans 4
F1 0.56; AUC 0.51 
Outlier Method 3; Standardise Method 4 Kmeans 5
F1 0.56; AUC 0.51 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Outlier Method 4; Standardise Method 4 Kmeans 2
F1 0.56; AUC 0.51 
Outlier Method 4; Standardise Method 4 Kmeans 3
F1 0.56; AUC 0.52 
Outlier Method 4; Standardise Method 4 Kmeans 4
F1 0.56; AUC 0.52 
Outlier Method 4; Standardise Method 4 Kmeans 5
F1 0.56; AUC 0.52 
Outlier Method 5; Standardise Method 4 Kmeans 2
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 4 Kmeans 3
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 4 Kmeans 4
F1 0.57; AUC 0.52 
Outlier Method 5; Standardise Method 4 Kmeans 5
F1 0.57; AUC 0.52 


In [48]:
# reducing our sample due to in balance between y2=1 and y2=0
for m in range(1,5):
    for j in range(1,6):
        x=pd.DataFrame.copy(ipo_data)
        res_hold_f1=[]
        res_hold_auc=[]
        for i in range(len(x.columns)): 
            if x.iloc[:,i].dtype == float:
                x.iloc[:,i]=standard(x.iloc[:,i],m)            
                if x.iloc[:,i].dtype == float:
                    outlier(x.iloc[:,i],j) 
        for k in range(2,6):
            for count in range(1,50):


                # Create K-means model and determine euclidian distances for each data point
                train=['C3x','C1','T3xbin','T4x','T5xbin','S1x','S2x','S3x','C2','T2','T1','Other','Manufacturing','Services']
                X=x[train]
                kmeans_model = KMeans(n_clusters=k)
                distances = kmeans_model.fit_transform(X)

                # Create scatter plot using labels from K-means model as color
                labels = kmeans_model.labels_


                x['labels'] = labels
                train.append('labels')


                logreg = LogisticRegression()
                #train=['C4','C6x','T4x','S2x']
                #train=['C3x','C4','T3x','T4x','S1x','S2x','C2']

                #train=['C3x','C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2']
                #train=['C4','C1','C7','C6x','T3x','T4x','T5x','S1x','S2x','S3x','C2','Construction','Agriculture, Forestry and Fishing','Finance, Insurance and Real Estate','Manufacturing','Mining','Services','Transportation, Communications, Electric, Gas and Sanitary service']

                X=x[train]

                y=x['Y1']
                rfe = RFE(logreg, 18)
                rfe = rfe.fit(X,y)

                logit_model=sm.Logit(x['Y1'],x[train])
                

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#                 oversample = pd.concat([X_train,y_train],axis=1)
#                 max_size = oversample['Y2'].value_counts().max()

#                 lst = [oversample]
#                 for class_index, group in oversample.groupby('Y2'):
#                     lst.append(group.sample(max_size-len(group), replace=True))
#                 X_train = pd.concat(lst)
#                 y_train=pd.DataFrame.copy(X_train['Y2'])
#                 del X_train['Y2']
                logreg = LogisticRegression()
                logreg.fit(X_train, y_train)
                y_pred = logreg.predict(X_test)

                kfold = model_selection.KFold(n_splits=5, random_state=7)
                modelCV = LogisticRegression()
                scoring = 'accuracy'
                results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
                logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
                #confusion_matrix = confusion_matrix(y_test, y_pred)
                res_hold_f1.append(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2])
                res_hold_auc.append(logit_roc_auc)
            print('Outlier Method {}; Standardise Method {} Kmeans {}'.format(j,m,k))
            print('F1 {:.2f}; AUC {:.2f} '.format(np.mean(res_hold_f1),np.mean(res_hold_auc)))
       #     if(precision_recall_fscore_support(y_test, y_pred, average='weighted')[2]>0.61 or logit_roc_auc>0.55):
               #   print('True')
                #result=logit_model.fit()
#                 print(rfe.support_)
#                 print(rfe.ranking_)
#                 print(result.summary())
#                 print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
#                 print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

#                 print(confusion_matrix)
#                 print(classification_report(y_test, y_pred))


#                 fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
#                 plt.figure()
#                 plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#                 plt.plot([0, 1], [0, 1],'r--')
#                 plt.xlim([0.0, 1.0])
#                 plt.ylim([0.0, 1.05])
#                 plt.xlabel('False Positive Rate')
#                 plt.ylabel('True Positive Rate')
#                 plt.title('Receiver operating characteristic')
#                 plt.legend(loc="lower right")
#                 plt.savefig('Log_ROC')


#                 plt.show()
#                 plt.scatter(distances[:,0], distances[:,1], c=labels)
#                 plt.title('Kmeans Clusters')
#                 plt.show()

                          

C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean-3*std
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean+3*std


Outlier Method 1; Standardise Method 1 Kmeans 2
F1 0.54; AUC 0.54 
Outlier Method 1; Standardise Method 1 Kmeans 3
F1 0.54; AUC 0.54 
Outlier Method 1; Standardise Method 1 Kmeans 4
F1 0.54; AUC 0.55 
Outlier Method 1; Standardise Method 1 Kmeans 5
F1 0.54; AUC 0.55 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Outlier Method 2; Standardise Method 1 Kmeans 2
F1 0.54; AUC 0.55 
Outlier Method 2; Standardise Method 1 Kmeans 3
F1 0.54; AUC 0.54 
Outlier Method 2; Standardise Method 1 Kmeans 4
F1 0.54; AUC 0.54 
Outlier Method 2; Standardise Method 1 Kmeans 5
F1 0.54; AUC 0.54 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


...

Outlier Method 2; Standardise Method 4 Kmeans 2
F1 0.53; AUC 0.54 
Outlier Method 2; Standardise Method 4 Kmeans 3
F1 0.53; AUC 0.54 
Outlier Method 2; Standardise Method 4 Kmeans 4
F1 0.53; AUC 0.54 
Outlier Method 2; Standardise Method 4 Kmeans 5
F1 0.53; AUC 0.54 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = mean
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = mean


Outlier Method 3; Standardise Method 4 Kmeans 2
F1 0.52; AUC 0.53 
Outlier Method 3; Standardise Method 4 Kmeans 3
F1 0.52; AUC 0.53 
Outlier Method 3; Standardise Method 4 Kmeans 4
F1 0.53; AUC 0.53 
Outlier Method 3; Standardise Method 4 Kmeans 5
F1 0.53; AUC 0.53 


C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_lower] = lq
C:\Users\conor\Documents\Fairfield College\IS540 Data Mining\Competitions\IS540-Code\Scratch Files\Model Version Control\outliers.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data[outlier_upper] = uq


Outlier Method 4; Standardise Method 4 Kmeans 2
F1 0.54; AUC 0.54 
Outlier Method 4; Standardise Method 4 Kmeans 3
F1 0.53; AUC 0.54 
Outlier Method 4; Standardise Method 4 Kmeans 4
F1 0.53; AUC 0.54 
Outlier Method 4; Standardise Method 4 Kmeans 5
F1 0.53; AUC 0.54 
Outlier Method 5; Standardise Method 4 Kmeans 2
F1 0.53; AUC 0.54 
Outlier Method 5; Standardise Method 4 Kmeans 3
F1 0.53; AUC 0.54 
Outlier Method 5; Standardise Method 4 Kmeans 4
F1 0.53; AUC 0.54 
Outlier Method 5; Standardise Method 4 Kmeans 5
F1 0.53; AUC 0.54 


In [50]:
print(np.mean(res_hold_f1))

0.532871838677


In [51]:
corr=['C1','C4','C6x','C7','T3x','T4x','T5x','S1x','S2x','S3x','T1','T2','Y1','Y2']
ipo_data[corr].corr(method='spearman').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,C1,C4,C6x,C7,T3x,T4x,T5x,S1x,S2x,S3x,Y1,Y2
C1,1.0,0.091,-0.064,0.053,-0.058,0.055,0.031,0.033,0.048,-0.026,0.07,0.0048
C4,0.091,1.0,0.093,0.018,0.057,0.085,0.062,0.0012,0.0041,0.039,-0.071,0.1
C6x,-0.064,0.093,1.0,-0.054,-0.039,-0.024,-0.098,-0.081,-0.12,-0.045,-0.77,0.24
C7,0.053,0.018,-0.054,1.0,-0.12,0.1,-0.11,0.087,0.19,-0.07,0.003,-0.054
T3x,-0.058,0.057,-0.039,-0.12,1.0,-0.11,0.45,0.19,0.074,0.37,0.11,-0.016
T4x,0.055,0.085,-0.024,0.1,-0.11,1.0,-0.086,0.071,-0.024,-0.091,0.073,-0.01
T5x,0.031,0.062,-0.098,-0.11,0.45,-0.086,1.0,0.18,0.16,0.41,0.12,-0.018
S1x,0.033,0.0012,-0.081,0.087,0.19,0.071,0.18,1.0,0.13,0.2,0.11,-0.042
S2x,0.048,0.0041,-0.12,0.19,0.074,-0.024,0.16,0.13,1.0,0.14,0.18,-0.1
S3x,-0.026,0.039,-0.045,-0.07,0.37,-0.091,0.41,0.2,0.14,1.0,0.062,0.046


In [52]:
for i in (['C3x','C1','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T1','T2']):
    grps = pd.unique(ipo_data.T5xbin.values)
    d_data = {grp:ipo_data[i][ipo_data.T5xbin == grp] for grp in grps}
        #run anova
    anova = stats.f_oneway(d_data[1], d_data[2], d_data[3], d_data[4])
    print('T5xbin v Variable {} Result {}'.format(i,anova))

T5xbin v Variable C3x Result F_onewayResult(statistic=13.307847567548935, pvalue=1.8942409425887808e-08)
T5xbin v Variable C1 Result F_onewayResult(statistic=3.4226178824382321, pvalue=0.016952951985385246)
T5xbin v Variable T4x Result F_onewayResult(statistic=1.3210449114113771, pvalue=0.2664185622307832)
T5xbin v Variable S1x Result F_onewayResult(statistic=9.7958262540703043, pvalue=2.4794524068946706e-06)
T5xbin v Variable S2x Result F_onewayResult(statistic=5.0084500006630988, pvalue=0.001929173799471005)
T5xbin v Variable S3x Result F_onewayResult(statistic=21.013943899440317, pvalue=5.0098360513451667e-13)
T5xbin v Variable C2 Result F_onewayResult(statistic=3.7344881993853756, pvalue=0.011093937410023685)
T5xbin v Variable C4 Result F_onewayResult(statistic=1.5011151116921551, pvalue=0.21302631170079697)
T5xbin v Variable C6x Result F_onewayResult(statistic=1.8358427063824205, pvalue=0.13927884881823963)
T5xbin v Variable C7 Result F_onewayResult(statistic=5.0912867129796044, p

In [53]:
for i in (['C3x','C1','T4x','S1x','S2x','S3x','C2','C4','C6x','C7','T3x','T1','T2']):
    grps = pd.unique(ipo_data.T3xbin.values)
    d_data = {grp:ipo_data[i][ipo_data.T3xbin == grp] for grp in grps}
        #run anova
    anova = stats.f_oneway(d_data[1], d_data[2], d_data[3], d_data[4])
    print('T3xbin v Variable {} Result {}'.format(i,anova))

T3xbin v Variable C3x Result F_onewayResult(statistic=3.6974254970392857, pvalue=0.011668610635687486)
T3xbin v Variable C1 Result F_onewayResult(statistic=1.6570488383517468, pvalue=0.17499844207527826)
T3xbin v Variable T4x Result F_onewayResult(statistic=3.2233223560342341, pvalue=0.022202399512796435)
T3xbin v Variable S1x Result F_onewayResult(statistic=7.0209519690892916, pvalue=0.00011821356732629749)
T3xbin v Variable S2x Result F_onewayResult(statistic=2.0649630324003825, pvalue=0.10356708770353737)
T3xbin v Variable S3x Result F_onewayResult(statistic=17.334766558817861, pvalue=7.4480224672478992e-11)
T3xbin v Variable C2 Result F_onewayResult(statistic=1.5102926564456729, pvalue=0.21059007835044813)
T3xbin v Variable C4 Result F_onewayResult(statistic=0.81880889192866968, pvalue=0.48369942351712147)
T3xbin v Variable C6x Result F_onewayResult(statistic=0.23502105150542343, pvalue=0.87197638244317854)
T3xbin v Variable C7 Result F_onewayResult(statistic=4.5572031225284926, pv